# * Adhoc : Target

## Parameter

In [53]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 19)

## ETL Process...

### Step 1 : Import Data Source

In [139]:
''' Rawdata '''


''' Target_DTAC_Sales_Y2024 '''
src_file = 'C:\Ruz\Pentaho\Jobs\Input\Target\Source\Ref-K.Voraphon\DTAC_Sales_Target\Target_DTAC_Sales_Y2024.xlsx'
src_sheet = 'Rawdata'
src_df = pd.read_excel(src_file, sheet_name=src_sheet, index_col=None) 
src_df['MONTH_KEY'] = src_df['DATA_MONTH'].str.upper()
src_df['H_AREA_KEY'] = src_df['AREA_CD']
# src_df = src_df.replace(np.nan, None)
# src_df.tail(3)

In [140]:
''' Master Data '''


''' DIM_TIME '''
dt_file = '../CFW/data/dim_time.csv'
dt_df = pd.read_csv(dt_file)
dt_df['MONTH_KEY'] = dt_df['MONTH_SHORT'].str.upper()
# dt_df.tail(3)


''' DIM_MOOC_AREA '''
mooc_file = '../CFW/data/dim_mooc_area.csv'
cols = ['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=cols)
# Create H level
mooc_h_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['HOP_HINT']!='True Corp')]
mooc_h_df = mooc_h_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT']].drop_duplicates()
mooc_h_df.dropna(how='all', inplace=True)
mooc_h_df['H_AREA_KEY'] = mooc_h_df['ORGID_H'].apply(lambda x: x if re.search(r'[a-z]', x, re.I) else int(x))
# mooc_h_df.tail(3)

In [141]:
''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)

mooc_h_df[mooc_h_df['ORGID_H'].str.contains('^0')]

,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_R,TDS_RGM_CODE,ORGID_H,HOP_HINT,H_AREA_KEY
91,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,046,"BKK : Lat Krabang, Nong Chok, Khlong Sam Wa",46
92,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,043,"BKK : Min Buri, Khan Na Yao, Bueng Kum",43
152,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,033,"BKK : Phra Khanong, Khlong Toei, Bang Na, Suan...",33
178,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,026,"BKK : Phaya Thai, Din Daeng, Huai Khwang",26
182,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,044,"BKK : Prawet, Bang Kapi, Saphan Sung",44
196,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),B002,BMA-West,050,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",50
204,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),B002,BMA-West,049,"BKK : Thon Buri, Khlong San, Rat Burana, Thung...",49
205,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,031,"BKK : Phra Nakhon, Dusit, Bang Rak, Sathon, Po...",31
206,BMA,GX2,Regional Management 2 (BMA-East),B001,BMA-East,037,"BKK : Vadhana, Pathum Wan, Ratchathewi",37
208,BMA,GX1,Deputy CGO & Regional Management 1 (BMA-West),B002,BMA-West,006,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka",6


In [56]:
# pd.merge(left, right, how="left", on=["key1", "key2"])

# df1 = pd.merge(src_df, dt_df, how='left', on=['MONTH_KEY'])
df1 = pd.merge(src_df, dt_df[['MONTH_KEY', 'TM_KEY_MTH']], how='left', on='MONTH_KEY')
df1.tail(3)

,DATA_DATE,DATA_MONTH,DATA_YEAR,METRIC_CD,METRIC_NAME,METRIC_VALUES,COMP_CD,VERSION,AREA_CD,ARE_DESC,AREA_TYPE,MONTH_KEY,TM_KEY_MTH
1570203,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505
1570204,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505
1570205,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505


In [58]:
# x1 = src_df.set_index('MONTH_KEY')
# x2 = dt_df[['MONTH_KEY', 'TM_KEY_MTH']].set_index('MONTH_KEY')

# df2 = x1.join(x2, how='left', on='MONTH_KEY')
# df2.tail(3)

## Generate Output file

In [135]:
''' Create "mooc_h_df.xlsx" '''

# mooc_h_df.to_excel(f'temp/mooc_h_df.xlsx', sheet_name='Data', index=False)
# print(f'\n   -> Generate "mooc_h_df.xlsx" successfully')


   -> Generate "mooc_h_df.xlsx" successfully
